In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyarrow as pa
import pyarrow.parquet as pq
import geopandas as gpd
import seaborn as sns

#Packages
import matplotlib.ticker as mtick
from scipy import stats
pd.set_option('display.max_columns', None)

from scipy.stats import skew, kurtosis

import pygris
from shapely.geometry import Polygon

import shapely

In [8]:
df = pd.read_parquet("C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/FimaNfipClaims.parquet.gzip")

## Fifth Dataset [censusBlockGroupFips and Tract NA (Missing together)]
## No geographic values missing, no shapefiles missing

In [9]:
df_geographic_unique = df[['state', 'reportedZipCode', 'countyCode', 'censusTract', 'censusBlockGroupFips', 'latitude', 'longitude', 'yearOfLoss']].drop_duplicates()

In [10]:
# Defining conditions
condition1 = (df_geographic_unique['latitude'].notna() & 
              df_geographic_unique['censusBlockGroupFips'].isna() & 
              df_geographic_unique['countyCode'].notna() & 
              df_geographic_unique['reportedZipCode'].notna())

# Filtering the dataframe based on the combined conditions
df_geographic_unique = df_geographic_unique[condition1]

In [11]:
df_geographic_unique.head()

,state,reportedZipCode,countyCode,censusTract,censusBlockGroupFips,latitude,longitude,yearOfLoss
207,FL,32569.0,12091.0,NaN,NaN,30.4,-86.7,1995
310,SC,29902.0,45013.0,NaN,NaN,32.4,-80.7,1994
312,SC,29902.0,45013.0,NaN,NaN,32.4,-80.7,1996
331,VT,5158.0,50025.0,NaN,NaN,43.1,-72.5,1999
332,VT,5158.0,50025.0,NaN,NaN,43.1,-72.5,2011


In [12]:
print(sum(df_geographic_unique['state'].isna()))
print(sum(df_geographic_unique['reportedZipCode'].isna()))
print(sum(df_geographic_unique['countyCode'].isna()))
print(sum(df_geographic_unique['censusTract'].isna()))
print(sum(df_geographic_unique['censusBlockGroupFips'].isna()))
print(sum(df_geographic_unique['latitude'].isna()))
print(sum(df_geographic_unique['yearOfLoss'].isna()))

0
0
0
32761
32761
0
0


In [13]:
df_geographic_unique['reportedZipCode'] = df_geographic_unique['reportedZipCode'].dropna().astype(int).astype(str)
df_geographic_unique['reportedZipCode'] = [zipcode.zfill(5) for zipcode in df_geographic_unique['reportedZipCode']]

df_geographic_unique['countyCode'] = [str(int(float(i))) for i in df_geographic_unique['countyCode']]
df_geographic_unique['countyCode'] = [censusBG.zfill(5) for censusBG in df_geographic_unique['countyCode']]

In [14]:
# # Define bins and labels for yearOfLoss_1980_2020
# bins_1980_2020 = [df_geographic_unique['yearOfLoss'].min(), 1990, 2000, 2010, 2020, df_geographic_unique['yearOfLoss'].max() + 1]
# labels_1980_2020 = [1980, 1990, 2000, 2010, 2020]

# df_geographic_unique['yearOfLoss_1980_2020'] = pd.cut(df_geographic_unique['yearOfLoss'], bins=bins_1980_2020, labels=labels_1980_2020, right=False).astype(int)

In [15]:
# Define bins and labels for yearOfLoss_1990_2021

bins_1990_2021 = [df_geographic_unique['yearOfLoss'].min(), 2000, 2010, 2020, df_geographic_unique['yearOfLoss'].max() + 1]
labels_1990_2021 = [1990, 2000, 2010, 2020]

df_geographic_unique['yearOfLoss_1990_2021'] = pd.cut(df_geographic_unique['yearOfLoss'], bins=bins_1990_2021, labels=labels_1990_2021, right=False).astype(int)

bins_2000_2021 = [df_geographic_unique['yearOfLoss'].min(), 2000, 2010, 2020, df_geographic_unique['yearOfLoss'].max() + 1]
labels_2000_2021 = [1990, 2000, 2010, 2020]

df_geographic_unique['yearOfLoss_2000_2021'] = pd.cut(df_geographic_unique['yearOfLoss'], bins=bins_2000_2021, labels=labels_2000_2021, right=False).astype(int)


df_geographic_unique = df_geographic_unique.drop_duplicates()

In [16]:
import pandas as pd

# Step 1: Define the bins.
# Adjusting the bins so that 2011 falls into the 2012 bin
custom_bins = [0, 2000, 2010, 2012] + list(range(2013, 2023)) + [2024]

# Step 2: Define the labels.
# Adjusting the labels to reflect the new bin structure
custom_labels = [0, 2000, 2010] + list(range(2012, 2023))  # The label 2022 will apply to both 2022 and 2023.

# Ensure the number of labels is one less than the number of bin edges.
assert len(custom_bins) == len(custom_labels) + 1, "Number of bin labels must be one fewer than the number of bin edges."

# Step 3: Apply the custom binning and create the 'zip_year_bin' column.
df_geographic_unique['zip_year_bin'] = pd.cut(
    df_geographic_unique['yearOfLoss'],
    bins=custom_bins,
    labels=custom_labels,
    right=False,  # Ensuring the rightmost edge is exclusive.
    include_lowest=True,  # Including the leftmost edge.
).astype(int)  # Converting the resulting categories to integers for convenience.

In [17]:
df_geographic_unique.head()

,state,reportedZipCode,countyCode,censusTract,censusBlockGroupFips,latitude,longitude,yearOfLoss,yearOfLoss_1990_2021,yearOfLoss_2000_2021,zip_year_bin
207,FL,32569,12091,NaN,NaN,30.4,-86.7,1995,1990,1990,0
310,SC,29902,45013,NaN,NaN,32.4,-80.7,1994,1990,1990,0
312,SC,29902,45013,NaN,NaN,32.4,-80.7,1996,1990,1990,0
331,VT,05158,50025,NaN,NaN,43.1,-72.5,1999,1990,1990,0
332,VT,05158,50025,NaN,NaN,43.1,-72.5,2011,2010,2010,2011


## Read shapefiles

In [18]:
states = pygris.states()

state_df = states[['STUSPS', 'NAME', 'geometry']]

Using the default year of 2021


In [19]:
#Checking if all states found in our dataset are in the US Census Bureau TIGER/Line and cartographic boundary shapefiles

unique_states = df_geographic_unique['state'].unique()
state_STUSPS_unique = state_df['STUSPS'].unique()

np.all(np.isin(unique_states, state_STUSPS_unique))

True

In [20]:
# Read the parquet file
df_read = pd.read_parquet("C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/lat_long_geometry.parquet.gzip")

# Convert the WKT strings back to geometries
lat_long_df = gpd.GeoDataFrame(df_read, geometry=df_read['geometry'].apply(lambda x: shapely.wkt.loads(x)))

In [21]:
chunk_size = 50000  # adjust based on your system's capabilities
chunks = [x for x in range(0, 300000, chunk_size)]

gdf_list = []

for start in chunks:
    end = start + chunk_size
    temp_df = pd.read_parquet(f"C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/zipcode_geometry_{start}_{end}.parquet.gzip")
    gdf_read = gpd.GeoDataFrame(temp_df, geometry=temp_df['geometry'].apply(lambda x: shapely.wkt.loads(x)))
    gdf_list.append(gdf_read)
    
# Concatenate all GeoDataFrames in the list into a single GeoDataFrame
zipcode_df = pd.concat(gdf_list, ignore_index=True)

In [22]:
# Read the parquet file
df_read = pd.read_parquet("C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/County_geometry.parquet.gzip")

# Convert the WKT strings back to geometries
County_df = gpd.GeoDataFrame(df_read, geometry=df_read['geometry'].apply(lambda x: shapely.wkt.loads(x)))



## Geometry Intersection creation

In [23]:
state_df.rename(columns={'geometry': 'geometry_state'}, inplace=True)
lat_long_df.rename(columns={'geometry': 'geometry_lat_long'}, inplace=True)
zipcode_df.rename(columns={'geometry': 'geometry_zipcode'}, inplace=True)
County_df.rename(columns={'geometry': 'geometry_county'}, inplace=True)


C:\Users\jorda\AppData\Local\Temp\ipykernel_12272\2800662510.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_df.rename(columns={'geometry': 'geometry_state'}, inplace=True)


In [24]:
df_geographic_unique.columns

Index(['state', 'reportedZipCode', 'countyCode', 'censusTract',
       'censusBlockGroupFips', 'latitude', 'longitude', 'yearOfLoss',
       'yearOfLoss_1990_2021', 'yearOfLoss_2000_2021', 'zip_year_bin'],
      dtype='object')

In [25]:
df_geographic_unique['yearOfLoss_2000_2021'].value_counts()

1990    26129
2000     4863
2010     1486
2020      283
Name: yearOfLoss_2000_2021, dtype: int64

In [26]:
# Filter for post 2000
# df_geographic_unique = df_geographic_unique[(df_geographic_unique['yearOfLoss_1980_2020']!=1980) & (df_geographic_unique['yearOfLoss_1980_2020']!=1980)]

df_geographic_unique = df_geographic_unique[df_geographic_unique['yearOfLoss_2000_2021']!=1990]

In [27]:
# Setting the multi-index on lat_long_df
lat_long_df.set_index(['latitude', 'longitude'], inplace=True)

# Mapping the values
df_geographic_unique['geometry_lat_long'] = df_geographic_unique.set_index(['latitude', 'longitude']).index.map(lat_long_df['geometry_lat_long'])

# Resetting the index of lat_long_df (optional, but good practice)
lat_long_df.reset_index(inplace=True)

In [28]:
# Initial mapping with multi-index
zipcode_df.set_index(['ZIPcode', 'year'], inplace=True)
df_geographic_unique['geometry_zipcode'] = df_geographic_unique.set_index(['reportedZipCode', 'zip_year_bin']).index.map(zipcode_df['geometry_zipcode'])

# Resetting the index of BG_df (return to multi-index)
zipcode_df.reset_index(inplace=True)

In [29]:
# Setting the multi-index on lat_long_df
state_df.set_index(['STUSPS'], inplace=True)

# Mapping the values
df_geographic_unique['geometry_state'] = df_geographic_unique.set_index(['state']).index.map(state_df['geometry_state'])

# Resetting the index of lat_long_df (optional, but good practice)
state_df.reset_index(inplace=True)

In [30]:
# Initial mapping with multi-index
County_df.set_index(['CountyID'], inplace=True)
df_geographic_unique['geometry_county'] = df_geographic_unique.set_index(['countyCode']).index.map(County_df['geometry_county'])

# Resetting the index of County_df (return to multi-index)
County_df.reset_index(inplace=True)

## Drop rows with missing shapefiles

In [31]:
df_geographic_unique = df_geographic_unique[(df_geographic_unique['geometry_county'].notna())
                              & (df_geographic_unique['geometry_zipcode'].notna())]

## Creating the intersections

In [32]:
import geopandas as gpd
from shapely.geometry import Polygon
from itertools import product
import warnings
warnings.filterwarnings("ignore")

In [33]:
len(df_geographic_unique)

5620

In [34]:
# Create an empty GeoDataFrame to store the intersection results
new_unit_df = gpd.GeoDataFrame(columns=['reportedZipCode', 'countyCode', 'censusTract',
                                       'censusBlockGroupFips', 'latitude', 'longitude', 'year','year_zipcode', 'state', 'geometry_zipcode',
                                       'geometry_county', 'geometry_lat_long','geometry_state',
                                         'cbgInconsistent', 'tractInconsistent', 'countyInconsistent', 'stateInconsistent', 'latlongInconsistent', 'multiple', 'noOverlap', 'oneWrong'])

# Iterate through each row in BG_df_1990 and each row in lat_long_df to find intersections
for idx_unit, row_unit in df_geographic_unique.iterrows():
    year = row_unit['yearOfLoss_1990_2021']
    year_zipcode = row_unit['zip_year_bin']
    bg_id = row_unit['censusBlockGroupFips']
    tract_id = row_unit['censusTract']
    county_id = row_unit['countyCode']
    county_geometry = row_unit['geometry_county']
    state = row_unit['state']
    state_geometry = row_unit['geometry_state']
    lat_long_geometry = row_unit['geometry_lat_long']
    lat = row_unit['latitude']
    long = row_unit['longitude']
    zipcode_geometry = row_unit['geometry_zipcode']
    zipcode = row_unit['reportedZipCode']
    
    # Compute intersection geometry
    #intersection_geometry = bg_geometry.intersection(lat_long_geometry).intersection(zipcode_geometry).intersection(county_geometry).intersection(state_geometry).intersection(tract_geometry)
    
    # First intersection
    intersection_1 = lat_long_geometry.intersection(zipcode_geometry)


    intersection_3 = intersection_1.intersection(county_geometry)

    # Fourth intersection
    intersection_geometry = intersection_3.intersection(state_geometry)


    
    if intersection_geometry is None or intersection_geometry.is_empty:
        new_unit_df = pd.concat([new_unit_df, pd.DataFrame({
            'reportedZipCode': [zipcode],
            'countyCode': [county_id],
            'censusTract': [tract_id],
            'censusBlockGroupFips': [bg_id],
            'latitude': [lat],
            'longitude': [long],
            'year': [year],
            'year_zipcode': [year_zipcode],
            'state': [state],
            'geometry_zipcode': [zipcode_geometry],
            'geometry_county': [county_geometry],
            'geometry_lat_long': [lat_long_geometry],
            'geometry_state': [state_geometry]
        })], ignore_index=True)

In [35]:
len(new_unit_df)

147

In [36]:
for idx_unit, row_unit in new_unit_df.iterrows():
    year = row_unit['year']
    bg_id = row_unit['censusBlockGroupFips']
    tract_id = row_unit['censusTract']
    county_id = row_unit['countyCode']
    county_geometry = row_unit['geometry_county']
    state = row_unit['state']
    state_geometry = row_unit['geometry_state']
    lat_long_geometry = row_unit['geometry_lat_long']
    lat = row_unit['latitude']
    long = row_unit['longitude']
    zipcode_geometry = row_unit['geometry_zipcode']
    zipcode = row_unit['reportedZipCode']

    intersection_geometry = zipcode_geometry.intersection(state_geometry).intersection(lat_long_geometry)

    if intersection_geometry is None or intersection_geometry.is_empty:
        new_unit_df.at[idx_unit,'countyInconsistent'] = 0
    else:
        new_unit_df.at[idx_unit,'countyInconsistent'] = 1

In [37]:
new_unit_df['countyInconsistent'].sum()

1

In [38]:
for idx_unit, row_unit in new_unit_df.iterrows():
    year = row_unit['year']
    bg_id = row_unit['censusBlockGroupFips']
    tract_id = row_unit['censusTract']
    county_id = row_unit['countyCode']
    county_geometry = row_unit['geometry_county']
    state = row_unit['state']
    state_geometry = row_unit['geometry_state']
    lat_long_geometry = row_unit['geometry_lat_long']
    lat = row_unit['latitude']
    long = row_unit['longitude']
    zipcode_geometry = row_unit['geometry_zipcode']
    zipcode = row_unit['reportedZipCode']

    intersection_geometry = zipcode_geometry.intersection(county_geometry).intersection(lat_long_geometry)

    if intersection_geometry is None or intersection_geometry.is_empty:
        new_unit_df.at[idx_unit,'stateInconsistent'] = 0
    else:
        new_unit_df.at[idx_unit,'stateInconsistent'] = 1

In [39]:
new_unit_df['stateInconsistent'].sum()

0

In [40]:
for idx_unit, row_unit in new_unit_df.iterrows():
    year = row_unit['year']
    bg_id = row_unit['censusBlockGroupFips']
    tract_id = row_unit['censusTract']
    county_id = row_unit['countyCode']
    county_geometry = row_unit['geometry_county']
    state = row_unit['state']
    state_geometry = row_unit['geometry_state']
    lat_long_geometry = row_unit['geometry_lat_long']
    lat = row_unit['latitude']
    long = row_unit['longitude']
    zipcode_geometry = row_unit['geometry_zipcode']
    zipcode = row_unit['reportedZipCode']

    intersection_geometry = zipcode_geometry.intersection(county_geometry).intersection(state_geometry)

    if intersection_geometry is None or intersection_geometry.is_empty:
        new_unit_df.at[idx_unit,'latlongInconsistent'] = 0
    else:
        new_unit_df.at[idx_unit,'latlongInconsistent'] = 1

In [41]:
new_unit_df['latlongInconsistent'].sum()

118

In [42]:
for idx_unit, row_unit in new_unit_df.iterrows():
    year = row_unit['year']
    bg_id = row_unit['censusBlockGroupFips']
    tract_id = row_unit['censusTract']
    county_id = row_unit['countyCode']
    county_geometry = row_unit['geometry_county']
    state = row_unit['state']
    state_geometry = row_unit['geometry_state']
    lat_long_geometry = row_unit['geometry_lat_long']
    lat = row_unit['latitude']
    long = row_unit['longitude']
    zipcode_geometry = row_unit['geometry_zipcode']
    zipcode = row_unit['reportedZipCode']
    
    intersection_geometry = lat_long_geometry.intersection(county_geometry).intersection(state_geometry)

    if intersection_geometry is None or intersection_geometry.is_empty:
        new_unit_df.at[idx_unit,'zipInconsistent'] = 0
    else:
        new_unit_df.at[idx_unit,'zipInconsistent'] = 1

In [43]:
new_unit_df.head()

,reportedZipCode,countyCode,censusTract,censusBlockGroupFips,latitude,longitude,year,state,geometry_zipcode,geometry_county,geometry_lat_long,geometry_state,cbgInconsistent,tractInconsistent,countyInconsistent,stateInconsistent,latlongInconsistent,multiple,noOverlap,oneWrong,zipInconsistent
0,98852,53007,NaN,NaN,48.2,-120.6,2000,WA,"POLYGON ((-120.94138 48.530144, -120.941046 48...","POLYGON ((-120.907425 48.163184, -120.907399 4...",POLYGON ((-120.64999999999999 48.1500000000000...,"MULTIPOLYGON (((-122.33164 48.020556, -122.328...",NaN,NaN,0,0,1,NaN,NaN,NaN,1.0
1,27968,37055,NaN,NaN,36.0,-75.6,2000,NC,"MULTIPOLYGON (((-75.485614 35.649355, -75.4856...","POLYGON ((-75.77358699999999 36.050824, -75.77...","POLYGON ((-75.64999999999999 35.95, -75.55 35....","MULTIPOLYGON (((-75.726807 35.935844, -75.7182...",NaN,NaN,0,0,1,NaN,NaN,NaN,1.0
2,71373,22041,NaN,NaN,30.6,-96.3,2010,LA,"POLYGON ((-91.77152 31.329504, -91.77094199999...","POLYGON ((-91.800598 32.223563999999996, -91.8...","POLYGON ((-96.35 30.55, -96.25 30.55, -96.25 3...","MULTIPOLYGON (((-88.8677 29.861551, -88.865659...",NaN,NaN,0,0,0,NaN,NaN,NaN,0.0
3,18827,42115,NaN,NaN,41.9,-75.6,2000,PA,"POLYGON ((-75.58524 41.96456, -75.585473 41.96...","POLYGON ((-75.782461 41.642848, -75.782652 41....","POLYGON ((-75.64999999999999 41.85, -75.55 41....","POLYGON ((-80.519891 40.906661, -80.5196264947...",NaN,NaN,0,0,1,NaN,NaN,NaN,1.0
4,52767,19163,NaN,NaN,41.5,-90.5,2000,IA,"MULTIPOLYGON (((-90.423531 41.559033, -90.4232...","POLYGON ((-90.899091 41.684182, -90.899067 41....","POLYGON ((-90.55 41.45, -90.45 41.45, -90.45 4...","POLYGON ((-96.6383589571896 42.735496024784, -...",NaN,NaN,0,0,1,NaN,NaN,NaN,1.0


In [44]:
new_unit_df['zipInconsistent'].sum()

144.0

In [45]:
mask = (new_unit_df['latlongInconsistent'] +  new_unit_df['zipInconsistent']+ new_unit_df['stateInconsistent'] + new_unit_df['countyInconsistent'] + new_unit_df['tractInconsistent'] + new_unit_df['cbgInconsistent']) > 1
new_unit_df.loc[mask, 'multiple'] = 1
new_unit_df.loc[~mask, 'multiple'] = 0

print(sum(new_unit_df['multiple']))

0


In [46]:
condition = (new_unit_df['latlongInconsistent'] + 
             new_unit_df['stateInconsistent'] + 
             new_unit_df['countyInconsistent'] + 
             new_unit_df['tractInconsistent'] + 
             new_unit_df['cbgInconsistent'] +
             new_unit_df['zipInconsistent']) == 0

new_unit_df['noOverlap'] = 0  # Default to 0
new_unit_df.loc[condition, 'noOverlap'] = 1

print(sum(new_unit_df['noOverlap']))

0


In [47]:
condition = (new_unit_df['latlongInconsistent'] + 
             new_unit_df['stateInconsistent'] + 
             new_unit_df['countyInconsistent'] + 
             new_unit_df['tractInconsistent'] + 
             new_unit_df['cbgInconsistent'] +
             new_unit_df['zipInconsistent']) == 1

new_unit_df['oneWrong'] = 0  # Default to 0
new_unit_df.loc[condition, 'oneWrong'] = 1

print(sum(new_unit_df['oneWrong']))


0


In [48]:
# Initialize the count dictionary for 'Wrong' values
wrong_counts = {
    'lat_long_geometry': 0,
    'geometry_state': 0,
    'county_geometry': 0,
    'tract_geometry': 0,
    'bg_geometry': 0,
    'zip_geometry': 0  # added for zip codes
}

# Iterate through the DataFrame
for idx_unit, row_unit in new_unit_df.iterrows():
    if row_unit['oneWrong'] == 1:
        if row_unit['latlongInconsistent'] == 1:
            new_unit_df.at[idx_unit, 'lat_long_geometry'] = 'Wrong'
            wrong_counts['lat_long_geometry'] += 1
        if row_unit['stateInconsistent'] == 1:
            new_unit_df.at[idx_unit, 'state_geometry'] = 'Wrong'
            wrong_counts['state_geometry'] += 1
        if row_unit['countyInconsistent'] == 1:
            new_unit_df.at[idx_unit, 'county_geometry'] = 'Wrong'
            wrong_counts['county_geometry'] += 1
        if row_unit['tractInconsistent'] == 1:
            new_unit_df.at[idx_unit, 'tract_geometry'] = 'Wrong'
            wrong_counts['tract_geometry'] += 1
        if row_unit['cbgInconsistent'] == 1:
            new_unit_df.at[idx_unit, 'bg_geometry'] = 'Wrong'
            wrong_counts['bg_geometry'] += 1
        if row_unit['zipInconsistent'] == 1:  # added for zip codes
            new_unit_df.at[idx_unit, 'zip_geometry'] = 'Wrong'
            wrong_counts['zip_geometry'] += 1

# Print the 'Wrong' counts
for category, count in wrong_counts.items():
    print(f"{category}: {count} 'Wrong' values")

lat_long_geometry: 0 'Wrong' values
geometry_state: 0 'Wrong' values
county_geometry: 0 'Wrong' values
tract_geometry: 0 'Wrong' values
bg_geometry: 0 'Wrong' values
zip_geometry: 0 'Wrong' values


In [49]:
new_unit_df['CBG_Zip_Mutually_Inconsistent'] = np.where(
    (new_unit_df['latlongInconsistent'] == 0) & 
    (new_unit_df['cbgInconsistent'] == 1) & 
    (new_unit_df['zipInconsistent'] == 1), 1, 0)

In [50]:
sum(new_unit_df['CBG_Zip_Mutually_Inconsistent'] == 1)

0

In [51]:
len(new_unit_df[(new_unit_df['latlongInconsistent'] == 1) &(new_unit_df['cbgInconsistent'] == 1) & ((new_unit_df['latlongInconsistent'] + 
             new_unit_df['stateInconsistent'] + 
             new_unit_df['countyInconsistent'] + 
             new_unit_df['tractInconsistent'] + 
             new_unit_df['cbgInconsistent'] +
             new_unit_df['zipInconsistent'])  == 2)])

0

In [52]:
columns_to_drop = [
     'geometry_zipcode',
    'geometry_county', 'geometry_state', 'geometry_lat_long'
]

# Drop the columns
new_unit_df = new_unit_df.drop(columns= columns_to_drop)

In [53]:
# Convert 'zipInconsistent' column to integer type.
new_unit_df['zipInconsistent'] = new_unit_df['zipInconsistent'].astype(int)

# Save to CSV without index.
new_unit_df.to_parquet(r'C:\Users\Asus\Box\Flood Damage PredictionProject\Dataset\InconsistencyDataframes\inconsistency_dataframe_5.parquet.gzip', compression = 'gzip', index=False)

In [54]:
len(new_unit_df)

147

In [55]:
# Check for duplicates
duplicates = new_unit_df[new_unit_df.duplicated(keep=False)]

print(duplicates)

    reportedZipCode countyCode  censusTract  censusBlockGroupFips  latitude  \
0             98852      53007          NaN                   NaN      48.2   
4             52767      19163          NaN                   NaN      41.5   
5             98852      53007          NaN                   NaN      48.2   
10            25152      54039          NaN                   NaN      38.2   
11            25152      54039          NaN                   NaN      38.2   
..              ...        ...          ...                   ...       ...   
134           18473      42127          NaN                   NaN      41.6   
141           52767      19163          NaN                   NaN      41.5   
142           52767      19163          NaN                   NaN      41.5   
143           38644      28027          NaN                   NaN      34.5   
144           38644      28027          NaN                   NaN      34.5   

     longitude  year state cbgInconsistent tractInc

In [56]:
# Convert 'zipInconsistent' column to integer type.
new_unit_df['zipInconsistent'] = new_unit_df['zipInconsistent'].astype(int)

# Save to CSV without index.
new_unit_df.to_parquet(r'C:\Users\Asus\Box\Flood Damage PredictionProject\Dataset\InconsistencyDataframes\inconsistency_dataframe_5.parquet.gzip', compression = 'gzip',  index=False)